## Optimization for deep learning

### 0. Gradient Descent

Gradient Descent는 `loss fn.`의 기울기를 이용해서 최저점을 찾고자하는 방법으로 다음과 같은 예를 통해서 설명이 가능하다. 

<img src='./imgs/gd.jpeg' width=500>
위의 그림을 보면, 기울기가 음수인 1, 3의 경우에 대해서는 기존의 가로축 좌표에서 오른쪽으로 향해야하고, 기울기가 양수인 2, 4의 경우에 대해서는 기존의 가로축 좌표에서 왼쪽으로 향해야합니다. 그렇기 때문에 다음과 같은 하나의 식으로 표현이 가능합니다.

$$ w_{t+1} = w_t - \gamma \times \nabla L(w_t)$$

그리고 여기서 $\gamma$가 `learning rate`로서 주어진 방향으로의 `step size`를 정해주는 파라미터입니다.

우리의 모델이 트레이닝 데이터셋 전체에 대해 loss를 구하고는 이렇게 weight vector들을 업데이트 하는 것입니다. 하지만 여기서 우리가 무심코 지나쳐서는 안 되는 부분이 있습니다. 바로 “트레이닝 데이터셋 전체에 대해 loss를 구하고는” 이 부분입니다. 이를 가장 기본적인 Batch gradient descent 또는 Gradient descent라고 합니다.

```python

for epoch in range(num_epochs):
    
    ...
    
    grads = get_gradient()
    for key in params.keys():
        params[key] -= lr*grads[key]
```

즉, 전체 데이터셋을 한 번 다 본 후에 `loss`를 구해서 이에 대한 미분값으로 `weights`를 업데이틑하는 방식입니다. 

> 이러한 Batch Gradient Descent의 단점은 모든 데이터 셋을 다 고려하면서 업데이트를 하기에는 시간이 굉장히 오래 걸리며, 이렇게 모든 데이터 셋을 `loss`가 다 고려할 수 없다는 점입니다. 그래서 나온 방법이 아래으 Stochastic Gradient Descent입니다.

### 1. SGD (Stochastic Gradient Descent)

#### 1-1. 
SGD는 앞선 BGD의 개선책으로서 데이터 셋을 모두 다보고 나서 `loss`를 계산하는 것이 아닌, 데이터를 1개를 보고 업데이트를 하는 방식입니다. 이를 전체 트레이닝 데이터셋에 있는 모든 데이터에 대해 1번씩 합니다. 그러면 트레이닝 데이터셋의 데이터 개수인 총 n번 update가 이루어집니다. 그리고는 데이터셋을 랜덤하게 섞습니다. 그리고는 또다시 트레이닝 데이터셋에서 1개씩 데이터를 보면서 update를 해나갑니다. 그러면 전체 데이터셋을 1바퀴 돌 때마다 랜덤하게 shuffle을 해주기 때문에 각기 다른 순서로 update가 이루어집니다. 이렇기 때문에 stochastic이라는 이름이 붙습니다. 


```python
for epoch in range(num_epochs):
    
    ...
    
    shuffle(train_dataloader)
    grads = get_gradient()
    for key in params.keys():
        params[key] -= lr*grads[key]
```


> 하지만, 저 수많은 데이터들을 1개 보고 업데이트 하고 1개 보고 업데이트 하는 것은 보기만 해도 지칩니다. 그래서 적절히 절충한 방법이 등장합니다. 바로 Mini-batch Gradient Descent입니다.




#### 1-2. Mini-batch Gradient Descent

데이터 B개(하나의 mini-batch)를 본 후에 update가 이루어지기 때문에 병렬처리가 가능합니다. Mini-batch간의 loss를 구하고는 평균을 내서 update를 하게 되는데, 잘 처리하면 이 각 mini-batch에 대해 병렬처리가 가능하여 GPU가 도움을 줄 수 있습니다. 요즘은 Mini-batch Gradient Descent가 굉장히 보편화되어서 SGD라는 용어가 Mini-batch Gradient Descent를 의미하는 경우가 많습니다. 

```python
for epoch in range(num_epochs):
    
    ...

    shuffle(train_dataloader)
	for one_batch in get_mini_batches(train_dataloader, one_batch_size=32):
		grads = get_gradient()
        for key in params.keys():
            params[key] -= lr*grads[key]```

>하지만, SGD는 여전히 작정 기울어진 방향으로 이동하는 방식이기 때문에 탐색경로가 매우 비효율적입니다. 또한 SGD는 방향에 따라서 기울기 값이 달라지는 경우에 대해서는 적합하지 않은데 최소값의 방향으로만 움직이기 때문에 본래의 최저점으로 기울기의 방향이 좀처럼 향하지 않게 되고 다른 방향으로 가기 때문에 더욱 비효율적입니다. 

<img src='./imgs/compr.png' width=500>

모든 데이터를 다 한꺼번에 고려한 후 1스텝을 가는 BGD가 SGD보다 더 최적의 루트(혹은 좋은 오솔길)로 산을 내려가게 됩니다. 그에 반해 결국 SGD도 적당한 곳으로 잘 내려가긴 하지만 다른 곳도 좀 헤매면서 내려갑니다. 여기서도 이미 SGD가 Mini-batch를 구현하고 있어서 update 횟수가 BGD보다 그렇게 많지 않습니다. 극단적인 예시를 위해 아래 그림을 보면 정의 그대로의 SGD가 얼마나 많은 업데이트를 하고 있는지 알 수 있습니다.

이처럼 SGD, Mini-batch GD가 굉장히 헤매면서 가는 것을 알 수 있습니다. 이 헤매는 것을 어떻게 하면 좀 줄일 수 있을까, local minimum이나 saddle point에 어떻게 하면 안주하지 않게 할 수 있을까 이런 질문들에서 다른 여러 optimizer들이 등장합니다.

결국, 우리는 다음과 같이 Gradient descent를 활용하여 신경망을 업데이트합니다.

$$ w_{t+1} = w_t - \gamma \times \nabla L(w_t)$$

이때, 파라미터가 될 수 있는 것, 즉 우리가 변화시켜 개선하는 데에 공헌할 수 있는 파라미터는 `learning rate`와 `gradient`가 됩니다. 이는 업데이트를 하고자 하는 크기(`step size`)와 방향이라고 할 수 있습니다. 그렇기 때문에 Gradient Descent의 발전도 마찬가지로 이 두 가지를 어떻게 효율적으로 도출하느냐에 따라 다음과 같이 이루어집니다. 

<img src='./imgs/optimizers.jpg'>

### 2. Momentum

즉, 관성과 같은 의미로서 weight를 업데이트를 할 때 이전에 내려왔던 방향도 반영을 해주자는 의미입니다. 

$$ w_{t+1} = w_t − \gamma \times \nabla L(w_t)$$

이렇게 기존 weight에 특정 방향의 vector( $−\gamma \times \nabla L(w_t)$) 만큼을 더해주고 있습니다. 그리고 momentum 방식은 단순히 지금 현재의 gradient를 update vector로 보자는 것이 아니라, 이전 step에서의 update vector도 고려하자는 의미입니다. 한 마디로 말해 이전에 움직였던 방향($v_t$)도 어느정도($\rho$) 고려해서 이번에 움직이자($−\gamma \times v_{t+1}$)는 의미입니다. 이를 수식으로 표현하면 아래와 같습니다.

$$ v_{t+1} = \rho v_t + \nabla L(w_t) $$
$$ w_{t+1} = w_t − \gamma \times v_{t+1} $$

이 식에서 눈여겨 봐야할 점은 이 momentum 식이 통계학에서 EMA(Exponential Moving Average) 또는 EWMA(Exponential Weighted Moving Average)로 불리는 형태로 표현될 수 있다는 점입니다. 우리나라 말로는 지수평균이라고 부르는 것 같습니다. $v_0 = 0$이라고 놓고 각 $v_n$을 다 더하는 꼴로 푼 다음 $\sum$으로 묶으면 이렇습니다. 

$$ v_t = \sum_{i=0}^t ρ^{t−i}\nabla L(w_i)$$

즉, 이는 기울기에서 속도의 개념이 추가되어 속도가 크게 나올수록 기울기가 크게 업데이트 되므로 기존 경사하강법의 단점을 보완하고자 합니다.


```python
grads = get_gradient() 
if self.v is None:
    self.v = {}
    for key, val in params.items():
        self.v[key] = np.zeros_like(val)
else:
    for key, val in params.items():
        self.v[key] = rho*self.v[key] + grads
        
for key in params.keys():
    self.v[key] = self.momentum*self.v[key]
```
        
이렇게 이전에 움직였던 방향도 고려를 해주는데 대개 $\rho$ 는 0.9 언저리의 값으로 합니다. 이전에 갔던 방향도 계속 더해주기 때문에 속력이 아닌 속도(velocity)가 계속 붙습니다. 공(ball) 비유로 들자면 공이 점점 내려가면서 속도가 붙습니다. 이런 점 덕분에 local minimum이나 saddle point에 걸리더라도 velocity로 그 영역을 탈출할 수 있습니다. 뿐만 아니라, 특정 방향으로 자주 update가 이루어진다면, 그 방향에 velocity가 생겨서 지그재그로 왔다갔다 하는 상황에서도 velocity 덕분에 오른쪽 아래 그림처럼 앞으로 나아가며 지그재그도 상쇄가 가능합니다.

<img src='./imgs/sgd_with_momentum.png' width=600>

>만약 굉장히 좁고 가파른 minima가 있을 경우엔 velocity 때문에 거기는 그냥 지나칠 수 있게 되지 않을까?
CS231n 강의에 따르면 사실 그런 좁고 가파른 minima에는 오히려 도달하는 것이 좋지 않을 수 있습니다. 왜냐하면 그런 좁고 가파른 minima에 도달하는 것은 overfitting이 될 수도 있기 때문입니다. 만약에 우리가 트레이닝셋을 2배로 늘려서 다시 학습을 할 경우에는 이 optimization 지형이 아예 바뀌어서 그때는 이 좁고 가파른 minima가 아예 사라져있을 수 있습니다. 좀 더 평평한 minima에 도달하는 것이 더 일반화 가능성도 높고, 탄탄할 것입니다.
이런 momentum 방법을 좀더 개선시킨 알고리즘도 있습니다. Nesterov라는 학자가 고안해낸 것에서 이름을 딴 Nesterov Momentum입니다.



### 3. NAG (Nesterov/Acceralte Gradient)

<img src='./imgs/na.png' width=600>

* plain momentum 

    1. 현재 위치에서의 gradient를 구한다
    2. 이전 velocity를 가져온다
    3. 이 두 가지 방향을 더한 방향으로 한 step 간다

* Nesterov momentum

    1. 이전 velocity를 가져온다
    2. 이전 velocity 방향대로만 갔을 때의 지점에서 gradient를 구한다
    3. 이 두 가지를 더한 방향으로 한 step 간다

momentum 방법에서는 현재 상황에서의 gradient로 velocity가 잘못된 방향으로 갔을 경우를 대비해 미리 예방하는 느낌이라면, Nesterov momentum 방법은 velocity로 잘못된 방향으로 간 후에 그 상황에서 그것을 gradient로 대처하는 느낌입니다. CS231n에서 말하기를 신경망과 같은 non-convex function에서는 효과가 그렇게 좋은지는 모르겠다고 합니다.

> 우리가 optimum에 다다르고 있는데 그냥 지나치게 되는 경우도 발생할 수 있습니다. 왜냐하면 local minimum에 다가가는 상황임에도 아무래도 learning rate가 늘 일괄적으로 적용되고 있기 때문입니다. 그래서 learning rate를 좀더 가변적으로 적용하자는 아이디어인 Adagrad가 나옵니다.

### 4. Adagrad

Adagrad라는 알고리즘은 한 마디로 요약하자면, 여태까지 많이 가보지 않았던 방향보다는 많이 갔던 방향에 대해 그 영향력을 점점 줄여나가는 방식입니다. 코드로 살펴보는 게 오히려 이해가 편할 수도 있겠습니다.

```python
current_gradient=evaluate_gradient(x)
grad_squared=grad_squared + current_gradient**2
weight= weight - learning_rate * current_gradient /(np.sqrt(grad_squared) + 1e-8)
```
우선 현재 위치에서의 기울기를 구하고 이를 (element-wise)제곱한 값을 grad_squared 에 계속 누적하는 식입니다. 어떻게 보면, grad_squared는 일종의 gradient 히스토리라고 볼 수 있겠습니다. 그리고는 learning_rate 를 이것의 제곱근으로 나눕니다. grad_squared 안에서는 여태까지 가파르던 방향(n차원 공간 속에서의 한 방향)은 값이 클 것이고, 완만했던 방향은 값이 작을 것입니다. 그렇기 때문에 이것으로 learning_rate 를 나눠주어 scale할 경우(작은 ϵ 을 더하여 분모가 0이 되는 것을 방지합니다), 분모의 크기에 따라 learning_rate도 다 달라집니다. 여태까지 가파르던 방향의 learning rate는 상대적으로 작아지고, 완만했던 방향은 learning rate는 상대적으로 커집니다. 이렇게 하는 데에 대한 직관은 원 논문 일부분을 발췌하여 강조 표시하였습니다.

>Informally, our procedures give frequently occurring features very low learning rates and infrequent features high learning rates, where the intuition is that each time an infrequent feature is seen, the learner should “take notice.” Thus, the adaptation facilitates finding and identifying very predictive but comparatively rare features.

위의 코드를 이제 수식으로 옮겨보겠습니다. 이제는 한결 수식이 더 잘 읽힙니다.

$$ G_{t+1} = G_t + (\nabla L(w_t))^2 $$ 
$$w_{t+1} = w_t − \frac{\gamma}{\sqrt{G_{t+1} + \epsilon}} \times \nabla L(w_t)$$

> 이 식을 따를 때, 만약 시간에 따라서 한 step의 크기는 어떻게 될지 생각해보겠습니다. G 는 계속 점점 커져만 갑니다. 이에 따라 step의 크기는 점점 작아집니다. 이런 성질이 convex function에서는 좋습니다. 왜냐하면 점점 minima에 가까워지면서 느려져야 하기 때문입니다. 그렇지만 non-convex한 경우에는 그다지 효과적이지 않을 수 있습니다. 왜냐하면 saddle point에 갇힐 수 있기 때문입니다. 이를 고치기 위해 우리 친숙한 Geoffrey Hinton이 만든 RMSProp알고리즘이 나옵니다. 덧붙여 이야기하자면, Adagrad는 learning rate를 다른 알고리즘보다는 좀 더 크게 주어야 진행이 너무 더디지 않습니다.

### 5. RMSProp

RMSProp(Root Mean Squared Prop)의 아이디어는 간단합니다. Gradient 히스토리도 마치 위에서 봤던 momentum처럼 decay하게 하자는 아이디어입니다. 이번에도 코드를 먼저 살펴보겠습니다.

```python
current_gradient=evaluate_gradient(x)
grad_squared=decay_rate * grad_squared + (1 - decay_rate) * current_gradient**2
weight= weight - learning_rate * current_gradient /(np.sqrt(grad_squared) + 1e-8)
```
위의 Adagrad 방법에서 딱 1줄 달라졌습니다. decay_rate을 두어 기존에는 그대로 누적되던 것들을 조금씩 decay하게 만듭니다. Hinton 교수에 의하면 decay_rate 는 0.9, learning_rate 는 0.001로 하는 것이 디폴트로 괜찮은 것 같답니다. RMSProp의 수식도 달라지는 것이 크게 없습니다. 

$$ G_{t+1} = \beta \times G_t + (1 - \beta)(\nabla L(w_t))^2 $$ 
$$w_{t+1} = w_t − \frac{\gamma}{\sqrt{G_{t+1} + \epsilon}} \times \nabla L(w_t)$$

### 6. Adam

흔히 어떤 두 개의 좋은 것이 있다면, 그 두 가지를 한 데 합치는 방식이 등장하곤 합니다. Optimizer도 예외는 아닙니다. 그렇게 momentum과 gradient 히스토리 모두를 고려하는 방식이 등장하였는데, 바로 Adam입니다. 이름에서부터 Adagrad + Momentum 이라는 느낌이 묻어납니다. 코드를 보겠습니다.

```python
first_moment=0
second_moment=0

for t in range(m):
	current_gradient=evaluate_gradient(x)

	first_moment=beta1*first_moment + (1-beta1)*current_gradient
	second_moment=beta2*second_moment +(1-beta2)*current_gradient**2

	unbiased_first=first_moment / (1-beta1**t)
	unbiased_second=second_moment / (1-beta2**t)

	weight=weight-learning_rate * unbiased_first / (np.sqrt(unbiased_second)+1e-8)

```

우선 7번째 줄은 우리가 앞서 봤던 momentum과 비슷합니다. 그리고 8번째 줄은 gradient 히스토리와 같은 식입니다. 이 둘을 적당히 버무린 것입니다. 그리고 각각 다른 파라미터인 beta1, beta2 를 주었습니다. 그런데 10,11번째 줄에 못 보던 코드가 들어있습니다. 이는 우리가 처음에 굉장히 큰 폭으로 step을 밟는 것을 방지하기 위함입니다. 왜냐하면 초반에 우리의 first_moment와 second_moment가 0에 가까운 값으로 bias되어 있을 것이기 때문입니다. 그리고는 이 unbiased된 것들로 실제 step을 갑니다. 왜 이렇게 되는지는 [원 논문](https://arxiv.org/pdf/1412.6980.pdf)에 나와있는데, 여기서 우리가 저 앞에 momentum을 EMA의 관점으로 바라보는 것이 필요합니다. 자세한 내용은 논문을 참조하기 바랍니다.

Adam은 굉장히 좋은 optimizer입니다. CS231n에서 Justin Jonhson도 말했듯이, 요즘 거의 디폴트로 사용되고 있는 optimizer입니다. 처음에 어떤 모델이든 학습을 시작해볼 때 시도하기에 좋은 수치값을 강의에서 말해주고 있을 정도입니다.

* beta1 = 0.9
* beta2 = 0.999
* learning_rate = 1e-3 or 5e-4



### Conclusion

<img src='./imgs/last.png'>
가운데의 optimum을 향해 여러 optimizer들이 달린 결과입니다. 보다시피 SGD가 가장 optimal route를 보여주고 있습니다. 하지만 SGD는 굉장히 연산량이 무겁고 힘들기 때문에 실전에서는 사용하기 힘듭니다. 그대신 나머지 3가지 optimizer들이 대안으로 존재합니다. SGD+Momentum과 Adam은 처음에는 optimum을 지나쳤다가(overshoot) 다시 돌아옵니다. 이는 알고리즘의 단점이 아니라 의도된 특성입니다. 다시 되짚어보면 이는 velocity로 인해 그런 것입니다. 그래야 sharp minima는 지나치고 local minima나 saddle point에 갇히지 않을 수 있습니다.

이 이외에도 NAdam이라 해서 앞서 봤던 Nesterov Momentum을 가미한 Adam이나, second-order optimization 기법들까지 다양하게 더 많습니다. 이렇게 여러 optimizer들이 존재하지만 디폴트로 가장 많이 쓰이는 것은 Adam입니다. 어떤 optimizer를 쓸지 모르겠다면 우선 Adam을 먼저 써보는 것을 추천합니다.

* Gradient Descent
    최적을 값을 찾아가는 것은 정확하지만 너무 느리다.

* Stochastic Gradient Descent를 통해서 빠르게 찾을 수 있도록 발전
    그러나 최적의 값을 찾아가는 방향이 뒤죽 박죽이고, 한 스텝 나아가기 위한 사이즈를 정하기 어렵다.위한 사이즈를 정하기 어렵다.

* 방향성
Momentum
NAG

* 스텝 사이즈
Adagrad
RMSPop
AdaDelta

* 방향성 + 스텝사이즈
Adam
Nadam

#### ref

https://www.slideshare.net/DavidKim486/neural-networks-basics-with-pytorch
https://hyunw.kim/blog/2017/11/01/Optimization.html#fn:1
https://www.slideshare.net/yongho/ss-79607172
https://seamless.tistory.com/38